## Cross-Bucket Tests on all 3 Fragment Types using Kalman Filter

### Part A: (1) Import Fragment Data from MATLAB, (2) Save Them in a .pickle file, and (3) Load Them In

###### (HOWEVER, YOU DON'T NEED TO LOAD THEM IN AGAIN SINCE STEP 1 ALREADY DOES THAT FOR YOU!)

In [1]:
## (Configuration) Allows you to return multiple variables from a single cell ##
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

## Allows you to import files from another folder in current directory ## 
import os 
import sys 
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

## Import standard package ###
import numpy as np
from scipy import io
import sys

# Specify Fragment Types to be used for this Anaylsis 
frag_type = ['AD', 'HV', 'VM'] # ['AccDec', 'HillValley', 'VelMin']

In [2]:
## (NEW) Import Data ##
# We'll load in position data and derive velocity and acceleration from it 

folder = '/Users/rbhatt6/Documents/MATLAB/' # For Windows: folder='C:\\Users\\rbhatt1\\Downloads\\' 

 #locals()["sortIn"+frag_type[i]] = io.loadmat(folder+'cleanedSortIn.mat')
for i in range(len(frag_type)):
    input = "sortIn_"+frag_type[i]
    locals()[input] = io.loadmat(folder + input + '.mat')

    output = "sortOut_" + frag_type[i]
    locals()[output] = io.loadmat(folder + output + '.mat')

    locals()[input] = np.squeeze(list(locals()[input].values())[3])
    locals()[output] = np.squeeze(list(locals()[output].values())[3])

### Part B: Preprocessing Decoder Data

In [3]:
#Import standard packages
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import io
from scipy import stats
import pickle
import sys

#Import metrics
from Neural_Decoding.metrics import get_R2
from Neural_Decoding.metrics import get_rho
from Neural_Decoding.metrics import get_R2_parts

#Import decoder functions
from Neural_Decoding.decoders import KalmanFilterDecoder

In [4]:
# (NEW)
#For the Kalman filter, we use the position, velocity, and acceleration as outputs.
#Ultimately, we are only concerned with the goodness of fit of velocity, but using them all as covariates helps performance.

for i in range(len(frag_type)):
    # Pulling local variables into new, temp variables
    output = locals()["sortOut_"+frag_type[i]]
    
    # Creating a local variable for each decoder_output
    decoder_output = "decoder_output_" + frag_type[i]
    locals()[decoder_output] = []

    for j in range(len(output)): # Number of buckets (i.e. 16 or 8)
        nFrags = output[j][0].shape[0]
        temp2 = []
        for k in range(0, nFrags, 1): # Number of fragments #output[0][0].shape[0]
            vel_X = float(output[j][0][k])
            vel_Y = float(output[j][1][k])
            acc_X = float(output[j][2][k])
            acc_Y = float(output[j][3][k])
            pos_X = float(output[j][4][k])
            pos_Y = float(output[j][5][k])
            temp = [vel_X, vel_Y, acc_X, acc_Y, pos_X, pos_Y]
            temp2.append(np.array(temp))
            #locals()[decoder_output][j][k].append(np.array(temp))
        #locals()[decoder_output][j].append(np.array(temp2))
        locals()[decoder_output].append(np.array(temp2))
        #temp = np.array(np.concatenate((vel_X, vel_Y, acc_X, acc_Y, outputX[j], outputY[j]),axis=1))


### Part C: Partitioning and Running the Kalman Filter on the Same Buckets 

In [5]:
#Set what part of data should be part of the training/testing/validation sets

training_range=[0, 0.8]
valid_range=[0.8,0.9]
testing_range=[0.9, 1]

In [6]:
# Running decoder on X and Y components together to get the prediction nom and denom to later compute the combined XY_FVAF
from Neural_Decoding.runModelsKF import run_model_kf

for i in range(len(frag_type)):

    # Pulling local variables 
    input = locals()["sortIn_"+frag_type[i]]
    output = locals()["decoder_output_"+frag_type[i]]

    # Creating a local variable to hold (X, Y) predicted outputs for each frag type AND their trained models
    parts = "pred_parts_" + frag_type[i]
    models = "models_" + frag_type[i]
    #locals()[parts] = run_model_kf(input, output, training_range, testing_range, valid_range, "parts", "within_bucket")
    R2s, trained_models = run_model_kf(input, output, training_range, testing_range, valid_range, "parts")
    locals()[parts] = R2s
    locals()[models] = trained_models

In [8]:
# Compute combined XY_FVAF (velocity only)
from Neural_Decoding.metrics import compute_XY_FVAF

for i in range(len(frag_type)):

    # Pulling previously computed predicted_parts (i.e. nom and denom)
    parts = locals()[ "pred_parts_" + frag_type[i]]
    
    # Creating a local variable to hold XY_FVAFs each frag type 
    XY_FVAF = "XY_FVAF_" + frag_type[i]
    locals()[XY_FVAF] = []

    for j in range(len(parts)):
        #curr_bucket = Kalman_R2s_combined[i]
        vel_x_nom = parts[j][0][0] # dim = (curr_bucket, nom, x_vel)
        vel_x_denom = parts[j][1][0] # dim = (curr_bucket, denom, x_vel)
        vel_y_nom = parts[j][0][1] # dim = (curr_bucket, nom, y_vel)
        vel_y_denom = parts[j][1][1] # dim = (curr_bucket, denom, y_vel)

        curr_bucket_XY_FVAF = compute_XY_FVAF(vel_x_nom,vel_x_denom,vel_y_nom,vel_y_denom)
        locals()[XY_FVAF].append(curr_bucket_XY_FVAF)

In [9]:
XY_FVAF_AD

[0.274861529126343,
 0.17285629320953566,
 0.3689220685199799,
 0.359915096119927,
 0.4215025371900333,
 0.37625623031487165,
 0.4259858030424114,
 0.3695129756493417,
 0.18764753306435566,
 0.20314343596785345,
 0.18883410343142448,
 0.0813192122389158,
 0.07460811664925293,
 0.14498036696841665,
 0.04125201098470266,
 0.13951618425674284]

In [31]:
# Testing cross-bucket function on AD fragments 
# pred_parts_ is actually the models now here
from Neural_Decoding.runModelsKF import cross_polarity_test, complete_opposite_bucket_test

ans3 = cross_polarity_test(pred_parts_AD, sortIn_AD, decoder_output_AD, training_range, testing_range, valid_range, "parts")
ans4 = complete_opposite_bucket_test(models_VM, sortIn_VM, decoder_output_VM, training_range, testing_range, valid_range, "parts", "VM")

In [7]:
# Testing cross-bucket function on AD fragments 
# pred_parts_ is actually the models now here
from Neural_Decoding.runModelsKF import cross_buckets_test

ans, ans2 = cross_buckets_test(pred_parts_AD, sortIn_AD, decoder_output_AD, training_range, testing_range, valid_range, "parts")

### Part D: Partitioning and Running the Kalman Filter on Separate Training and Test Buckets

In [11]:
# Cross-bucket test for all fragment types 
from Neural_Decoding.runModelsKF import cross_buckets_test

for i in range(len(frag_type)):

    # Pulling local variables 
    input = locals()["sortIn_" + frag_type[i]]
    output = locals()["decoder_output_" + frag_type[i]]
    models = locals()["models_" + frag_type[i]]

    # Creating a local variable to hold cross_bucket FVAFs (moreso, total-residual for each cross-bucket test)
    cross_buckets = "cross_buckets_FVAFs_" + frag_type[i]
    R2s, total_res = cross_buckets_test(models, input, output, training_range, testing_range, valid_range, "parts")
    locals()[cross_buckets] = total_res

In [33]:
# Cross-polarity and Complete-opposite tests for all fragment types 
from Neural_Decoding.runModelsKF import cross_buckets_test, cross_polarity_test, complete_opposite_bucket_test

for i in range(len(frag_type)):

    # Pulling local variables 
    input = locals()["sortIn_" + frag_type[i]]
    output = locals()["decoder_output_" + frag_type[i]]
    models = locals()["models_" + frag_type[i]]

    # Creating a local variable to hold cross-polarity and complete-opposite FVAFs
    cross_polarity = "cross_polarity_FVAFs_" + frag_type[i]
    locals()[cross_polarity] = cross_polarity_test(models, input, output, training_range, testing_range, valid_range, "parts", frag_type[i])
    complete_opposite = "complete_opposite_FVAFs_" + frag_type[i]
    locals()[complete_opposite] = complete_opposite_bucket_test(models, input, output, training_range, testing_range, valid_range, "parts", frag_type[i])

### Part E: Save all outputs (i.e. FVAFs)

In [38]:
# Saving results in a matlab file 
from scipy.io import savemat

# result1 = [XY_FVAF_AD, XY_FVAF_HV, XY_FVAF_VM]
# result2 = [cross_buckets_FVAFs_AD, cross_buckets_FVAFs_HV, cross_buckets_FVAFs_VM]
result3 = [cross_polarity_FVAFs_AD, cross_polarity_FVAFs_HV, cross_polarity_FVAFs_VM]
result4 = [complete_opposite_FVAFs_AD, complete_opposite_FVAFs_HV, complete_opposite_FVAFs_VM]

# FrameStack1 = np.empty((3,), dtype=object)
# FrameStack2 = np.empty((3,), dtype=object)
FrameStack3 = np.empty((3,), dtype=object)
FrameStack4 = np.empty((3,), dtype=object)
for i in range(len(result3)):
    # FrameStack1[i] = result1[i]
    # FrameStack2[i] = result2[i]
    FrameStack3[i] = result3[i]
    FrameStack4[i] = result4[i]
# savemat("XY_FVAF_All_Frags.mat", {"XY_FVAF":FrameStack1})
# savemat("Cross_Bucket_All_Frags.mat", {"Cross_Bucket":FrameStack2})
savemat("Cross_Polarity_All_Frags.mat", {"Cross_Polarity":FrameStack3})
savemat("Complete_Opposite_All_Frags.mat", {"Complete_Opposite":FrameStack4})

### Notes

In [ ]:
for i in range(len(sortIn_AD)):
    X_kf = sortIn_AD[i]
    y_kf = decoder_output_AD[i]
    num_examples_kf=X_kf.shape[0] # nRows (b/c nCols = number of units)
    
    #Note that each range has a buffer of 1 bin at the beginning and end
    #This makes it so that the different sets don't include overlapping data
    training_set=np.arange(int(np.round(training_range[0]*num_examples_kf))+1,int(np.round(training_range[1]*num_examples_kf))-1)
    testing_set=np.arange(int(np.round(testing_range[0]*num_examples_kf))+1,int(np.round(testing_range[1]*num_examples_kf))-1)
    valid_set=np.arange(int(np.round(valid_range[0]*num_examples_kf))+1,int(np.round(valid_range[1]*num_examples_kf))-1)

    #Get training data
    X_kf_train=X_kf[training_set,:]
    y_kf_train=y_kf[training_set,:]

    #Get testing data
    X_kf_test=X_kf[testing_set,:]
    y_kf_test=y_kf[testing_set,:]

    #Get validation data
    X_kf_valid=X_kf[valid_set,:]
    y_kf_valid=y_kf[valid_set,:]

    #Z-score inputs 
    X_kf_train_mean=np.nanmean(X_kf_train,axis=0)
    X_kf_train_std=np.nanstd(X_kf_train,axis=0)
    X_kf_train=(X_kf_train-X_kf_train_mean)/X_kf_train_std
    X_kf_test=(X_kf_test-X_kf_train_mean)/X_kf_train_std
    X_kf_valid=(X_kf_valid-X_kf_train_mean)/X_kf_train_std

    #Zero-center outputs
    y_kf_train_mean=np.mean(y_kf_train,axis=0)
    y_kf_train=y_kf_train-y_kf_train_mean
    y_kf_test=y_kf_test-y_kf_train_mean
    y_kf_valid=y_kf_valid-y_kf_train_mean

    #Declare model
    model_kf=KalmanFilterDecoder(C=1) #There is one optional parameter (see ReadMe)

    #Fit model
    model_kf.fit(X_kf_train,y_kf_train)

    #Get predictions
    y_valid_predicted_kf=model_kf.predict(X_kf_valid,y_kf_valid)

    #Get metrics of fit (see read me for more details on the differences between metrics)
    #First I'll get the R^2
    R2_kf=get_R2(y_kf_valid,y_valid_predicted_kf)
    print('R2:',R2_kf[0:2]) #I'm just printing the R^2's of the 1st and 2nd entries that correspond to the positions
    #Next I'll get the rho^2 (the pearson correlation squared)